In [84]:
#package installs
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import unidecode
import re

In [102]:
def statScraper(name, year, playerindex, team, team_ids):
    y = 0
    while True:
        playerindexint = int(playerindex) + y
        playerindex = '0' + str(playerindexint)
        for x in range(2):
            url, advanced = urlConvertor(name, year, playerindex, x)
            print(url)
            try:
                result = requests.get(url)
                print(result.status_code)
                src = result.content
                soup = BeautifulSoup(src, 'lxml')
                table = soup.find('table', {'class': 'row_summable'})
                trs = table.find_all('tr')
                rows = []
                for tr in trs:
                    tds = tr.find_all('td')
                    row = [td.text.replace('\n', '').strip() for td in tds]
                    rows.append(row)
                if x == 0:
                    columns = ['G', 'Date', 'Age', 'Tm', 'Location', 'Opp',
                               'GS', 'Active', 'MP', 'FG', 'FGA', 'FG%',
                               '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
                               'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
                               'PTS', 'GmSc', '+/-']
                    df = pd.DataFrame(rows, columns=columns)
                    df = df[df['Tm'] == team_ids[team][0]]
                    result.close()
                    df['PlayerID'] = name[2]
                    df['TmID'] = team_ids[team][1]
                elif x == 1:
                    columns = ['G', 'Date', 'Age', 'Tm', 'Location', 'Opp', 'GS',
                               'Active', 'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%',
                               'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
                               'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM']
                    df2 = pd.DataFrame(rows, columns=columns)
                    df2 = df2[df2['Tm'] == team_ids[team][0]]
                    result.close()
                    df2 = df2.drop(df2.columns[[0, 2, 3, 4, 5, 6, 7, 8, -2]], axis=1)
            except:
                pass
        try:
            if len(df) > 0:
                statdf = pd.merge(df, df2,how='outer', on='Date')
                statdf.insert(0, 'Name', name[1] + ', ' + name[0])
                print('Successfully Scraped:', name[0], name[1])
                return statdf
        except:
            if y > 4:
                print('Stat scraping failed for:', name[0], name[1])
                return None
            y+=1



In [86]:
def urlConvertor(name, year, playerindex, advanced: bool):
    lname = name[1]
    fname = name[0]
    lnamelist = list(lname)
    linitial = lnamelist[0]
    fnamelist = list(fname)
    lname5 = ''
    for x in range(5):
        try:
            lname5 += lnamelist[x]
        except:
            break
    f2i = fnamelist[0] + fnamelist[1]
    if advanced == False:
        url = f'https://www.basketball-reference.com/players/{linitial}/{lname5}{f2i}{playerindex}/gamelog/{year}/'
    else:
        url = f'https://www.basketball-reference.com/players/{linitial}/{lname5}{f2i}{playerindex}/gamelog-advanced/{year}/'
    return url, advanced


In [87]:
def rosterScraper(TeamID, year, id_number):
    url = f"https://www.basketball-reference.com/teams/{TeamID}/{year}.html"
    result = requests.get(url)
    try:
        src = result.content
        soup = BeautifulSoup(src, 'lxml')
        table = soup.find('table', {'class': 'sortable'})
        names = []
        trs = table.find_all('tr')
        rows = []
        for tr in trs:
            try:
                td = tr.find('td', {'data-stat': 'player'})
                a_tag = td.find('a')
                name = a_tag.get_text()
                name = unidecode.unidecode(name)
                name = re.sub(r'[^\w\s]','',name)
                names.append(name)
            except:
                pass
        for x in range(len(names)):
            names[x] = names[x].split()
            names[x].append(id_number)
            id_number += 1
        print(TeamID,"roster successfully parsed")
        return names
    except:
        return False



In [88]:
def teamFinder(year):
    team_id = 101
    url = f"https://www.basketball-reference.com/leagues/NBA_{year}.html"
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'lxml')
    table = soup.find('table', {'class': 'stats_table sortable', 'data-cols-to-freeze': ',2', 'id': 'per_game-team'})
    teams = []
    ids = {}
    trs = table.find_all('tr')
    x = 0
    for tr in trs[1:]:
        #print(tr.prettify())
        try:
            td = tr.find('td', {'data-stat': 'team', 'class': 'left'})
            a_tag = td.find('a')
            team = a_tag.get_text()
            teams.append(team)
            link = a_tag.attrs['href']
            link = list(link)
            acronym = link[7] + link[8] + link[9]
            ids[teams[x]] = []
            ids[teams[x]].append(acronym)
            x+=1
        except:
            pass
    for x in range(len(teams)):
        ids[teams[x]].append(team_id)
        team_id+=1
    return teams, ids

In [89]:
def leagueRosters(year):
    teamrosters = {}
    id_number = 1001
    teams, team_ids = teamFinder(year)
    for team in teams:
        teamrosters[team] = rosterScraper(team_ids[team][0], year, id_number)
        id_number+=len(teamrosters[team])
    return teamrosters, team_ids

In [90]:
def teamRoster(team, year):
    roster = []
    id_number = 1001
    teams, team_ids = teamFinder(year)
    roster = rosterScraper(team_ids[team][0], year, id_number)
    return roster, team_ids


In [91]:
def playerScraper(name, team, year):
    name = name.split()
    name.append('1001')
    teams, team_ids = teamFinder(year)
    df = statScraper(name, year, '01', team, team_ids)
    return df


In [92]:
def teamScraper(team, year):
    roster, team_ids = teamRoster(team, year)
    for x in range(len(roster)):
        try:
            if x == 0:
                player = roster[x]
                df = statScraper(player, year, '01', team, team_ids)
            else:
                player = roster[x]
                df2 = statScraper(player, year, '01', team, team_ids)
                df = df.append(df2)
        except:
            pass
    return df

In [93]:
def leagueScraper(year):
    rosters, team_ids = leagueRosters(year)
    y = 0
    for team in rosters:
        for x in range(len(rosters[team])):
            try:
                if x == 0 and y == 0:
                    player = rosters[team][x]
                    print(player)
                    df = statScraper(player, year, '01', team, team_ids)
                else:
                    player = rosters[team][x]
                    df = df.append(statScraper(player, year, '01', team, team_ids))
            except:
                pass
        y+=1
    return df

In [94]:
def save_df(df, title, path):
    df.to_csv(rf'{path}\{title}.csv', index=False)


In [103]:
save_df(teamScraper('Atlanta Hawks', '2016'), 'Hawks2016v2', r"C:\Users\jleus\OneDrive\Documents\CSVS")

ATL roster successfully parsed
https://www.basketball-reference.com/players/B/BazemKe01/gamelog/2016/
200
https://www.basketball-reference.com/players/B/BazemKe01/gamelog-advanced/2016/
200
Successfully Scraped: Kent Bazemore
https://www.basketball-reference.com/players/H/HardaTi01/gamelog/2016/
200
https://www.basketball-reference.com/players/H/HardaTi01/gamelog-advanced/2016/
200
https://www.basketball-reference.com/players/H/HardaTi02/gamelog/2016/
200
https://www.basketball-reference.com/players/H/HardaTi02/gamelog-advanced/2016/
200
Successfully Scraped: Tim Hardaway
https://www.basketball-reference.com/players/H/HinriKi01/gamelog/2016/
200
https://www.basketball-reference.com/players/H/HinriKi01/gamelog-advanced/2016/
200
Successfully Scraped: Kirk Hinrich
https://www.basketball-reference.com/players/H/HolidJu01/gamelog/2016/
200
https://www.basketball-reference.com/players/H/HolidJu01/gamelog-advanced/2016/
200
Successfully Scraped: Justin Holiday
https://www.basketball-referenc

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\jleus\\OneDrive\\Documents\\CSVS\\Hawks2016v2.csv'

In [101]:
statScraper(['Tim', 'Hardaway', 1002], '2016', '01', 'Atlanta Hawks', {'Golden State Warriors': ['GSW', 101], 'Oklahoma City Thunder': ['OKC', 102], 'Sacramento Kings': ['SAC', 103], 'Houston Rockets': ['HOU', 104], 'Boston Celtics': ['BOS', 105], 'Portland Trail Blazers': ['POR', 106], 'Los Angeles Clippers': ['LAC', 107], 'Cleveland Cavaliers': ['CLE', 108], 'Washington Wizards': ['WAS', 109], 'San Antonio Spurs': ['SAS', 110], 'Charlotte Hornets': ['CHO', 111], 'Atlanta Hawks': ['ATL', 112], 'Toronto Raptors': ['TOR', 113], 'New Orleans Pelicans': ['NOP', 114], 'Minnesota Timberwolves': ['MIN', 115], 'Dallas Mavericks': ['DAL', 116], 'Indiana Pacers': ['IND', 117], 'Orlando Magic': ['ORL', 118], 'Detroit Pistons': ['DET', 119], 'Denver Nuggets': ['DEN', 120], 'Chicago Bulls': ['CHI', 121], 'Phoenix Suns': ['PHO', 122], 'Miami Heat': ['MIA', 123], 'Memphis Grizzlies': ['MEM', 124], 'Milwaukee Bucks': ['MIL', 125], 'Brooklyn Nets': ['BRK', 126], 'New York Knicks': ['NYK', 127], 'Utah Jazz': ['UTA', 128], 'Philadelphia 76ers': ['PHI', 129], 'Los Angeles Lakers': ['LAL', 130]})


['Tim', 'Hardaway', 1002]
01
https://www.basketball-reference.com/players/H/HardaTi01/gamelog/2016/
200
https://www.basketball-reference.com/players/H/HardaTi01/gamelog-advanced/2016/
200
['Tim', 'Hardaway', 1002]
02
https://www.basketball-reference.com/players/H/HardaTi02/gamelog/2016/
200
https://www.basketball-reference.com/players/H/HardaTi02/gamelog-advanced/2016/
200
Successfully Scraped: Tim Hardaway


,Name,G,Date,Age,Tm,Location,Opp,GS,Active,MP,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,"Hardaway, Tim",,2015-10-27,23-225,ATL,,DET,L (-12),Inactive,None,...,None,None,None,None,None,None,None,None,None,None
1,"Hardaway, Tim",,2015-10-29,23-227,ATL,@,NYK,W (+11),Did Not Play,None,...,None,None,None,None,None,None,None,None,None,None
2,"Hardaway, Tim",,2015-10-30,23-228,ATL,,CHO,W (+3),Did Not Play,None,...,None,None,None,None,None,None,None,None,None,None
3,"Hardaway, Tim",,2015-11-01,23-230,ATL,@,CHO,W (+2),Inactive,None,...,None,None,None,None,None,None,None,None,None,None
4,"Hardaway, Tim",,2015-11-03,23-232,ATL,@,MIA,W (+6),Did Not Play,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,"Hardaway, Tim",47,2016-04-05,24-020,ATL,,PHO,W (+13),0,13:39,...,7.3,3.5,22.0,0.0,0.0,0.0,19.9,138,96,1.6
78,"Hardaway, Tim",48,2016-04-07,24-022,ATL,,TOR,W (+8),0,27:34,...,4.0,2.0,0.0,1.9,5.5,0.0,12.4,136,95,8.8
79,"Hardaway, Tim",49,2016-04-09,24-024,ATL,,BOS,W (+11),0,11:23,...,0.0,0.0,0.0,0.0,0.0,0.0,9.4,125,114,-7.5
80,"Hardaway, Tim",50,2016-04-11,24-026,ATL,@,CLE,L (-15),0,17:02,...,13.7,6.3,10.2,0.0,0.0,0.0,20.8,104,119,-1.5
